# 'Risk Management Dashboard'

Used 'dash' library in python.

If your labtop does not installed dash and plotly, you need install first to see and use this dashboard.

(1) Open new terminal and install 'dash', 'plotly'


(2) pip install both in python


(3) use with these libraries

We can check the results in dashboard whatever we want to check.

I used the cleaned data which was made by Sam.

Please check and feel free to feedback and point out anythings.


-----

### Refered definition of major columns

1. External Risk Estimate: This represents the loan applicant's external credit score. High scores can indicate high credit confidence.

2. NumSatisfactoryTrades: The number of transactions the lender is satisfied with. The higher this value, the more positive the lender's financial activities are.

3. PercentTradesNeverDelq: Percentage of transactions with no delinquency history. A high value may indicate that the borrower has a good credit history.

4. RecentDelq_1yr, RecentDeql_2yr: An indicator of delinquency in the last one and two years. The lower these values, the higher the borrower's credit confidence.

5. NumTotalTrades: Indicates the total number of transactions that the lender has.

6. NetFractionRevolvingBurden: Indicates the net burden on the revolving balance. The lower this value, the more stable the lender's financial status may look.

7. NumBank2NatlTradesWHighUtilization: The number of high utilization transactions between banks and nationwide. The more high utilization transactions, the higher the financial burden on borrowers.

8. PercentTradesWBalance: The percentage of transactions that have a balance in the transaction. The higher this value, the more active the lender's financial transactions.


-----

In [85]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_table

# Load cleaned(pre-processing) data

df = pd.read_csv('cleaned.csv')

# Create Dash app for the dashboard

app = dash.Dash(__name__)

# Define app layout (UI)

app.layout = html.Div([
    html.H1("HELOC loan Riskmanagement Dashboard",
            style={
                'textAlign': 'center',
                'color': '#503D36',
                'font-size': 40,
                'background-color': '#F9F9F9'
            }),
    
    # Add a division

    html.Div([
        dcc.Dropdown(
            id='Choosed-Column_1',
            options=[{'label': col, 'value': col} for col in df.columns],
            value='ExternalRiskEstimate',
            multi=False
        ),
        dcc.Dropdown(
            id='Choosed-Column_2',
            options=[{'label': col, 'value': col} for col in df.columns],
            value='NumSatisfactoryTrades',
            multi=False
        ),
        dcc.Dropdown(
            id='Choosed-Column_3',
            options=[{'label': col, 'value': col} for col in df.columns],
            value='PercentTradesNeverDelq',
            multi=False
        ),
        dcc.Dropdown(
            id='Choosed-Column_operator',
            options=[
                {'label': 'Greater value than', 'value': 'greater'},
                {'label': 'Less value than', 'value': 'less'}
            ],
            value='greater'
        ),
        dcc.Input(id='filter-value', type='number', value=0),
    ]),

# Add a division
    html.Div([
        dash_table.DataTable(
            id='filtered-data-table',
            columns=[{'name': col, 'id': col} for col in df.columns],
            page_size=10
        )
    ]),

# Add a division
    html.Div([
        dcc.Graph(
            id='scatter-plot'
        )
    ])
])

# Define callback to update graph
@app.callback(
    Output('filtered-data-table', 'data'),
    Output('scatter-plot', 'figure'),
    Input('Choosed-Column_1', 'value'),
    Input('Choosed-Column_2', 'value'),
    Input('Choosed-Column_3', 'value'),
    Input('Choosed-Column_operator', 'value'),
    Input('filter-value', 'value')
)
def update_filtered_data(col1, col2, col3, selected_operator, selected_value):
    try:
        selected_value = float(selected_value)
    except (ValueError, TypeError):
        return [], {}

    # Data filtering
    if selected_operator == 'greater':
        filtered_data = df[
            (df[col1] > selected_value) &
            (df[col2] > selected_value) &
            (df[col3] > selected_value)
        ]
    elif selected_operator == 'less':
        filtered_data = df[
            (df[col1] < selected_value) &
            (df[col2] < selected_value) &
            (df[col3] < selected_value)
        ]
        
# Scatter plot creation
    scatter_figure = px.scatter(
        filtered_data,
        x=col1,
        y=col2,
        color=col3,
        title='Scatter Plot'
    )

    return filtered_data.to_dict('records'), scatter_figure

if __name__ == '__main__':
    app.run_server(debug=True)
